[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/repos-especializacion-UdeA/estadistica/blob/main/trabajo3/trabajo3.ipynb)

In [37]:
# Bibliotecas necesarias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

# We will use some methods from the sklearn module
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import normalize

import scipy.stats as stats
from scipy.stats import bartlett, shapiro
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor
from IPython.display import display, Markdown

# Trabajo 3

## Base de datos

En un estudio a gran escala realizado en EE.UU sobre la eficacia en el control de infecciones hospitalarias se recogió información en 113 hospitales. A su equipo de trabajo le corresponde analizar una muestra aleatoria de n hospitales, que están dentro de un archivo de texto adjunto, donde n es el número de registros en el archivo asignado y X es el número de equipo asignad. La base de datos contiene las siguientes columnas (variables):
* **Y**: Riesgo de infección Probabilidad promedio estimada de adquirir infección en el hospital (en porcentaje).
* **$X_1$**: Duración de la estadía Duración promedio de la estadía de todos los pacientes en el hospital (en días).
* **$X_2$**: Rutina de cultivos Razón del número de cultivos realizados en pacientes sin síntomas de infección hospitalaria,
por cada 100.
* **$X_3$**: Número de camas Número promedio de camas en el hospital durante el periodo del estudio.
* **$X_4$**: Censo promedio diario Número promedio de pacientes en el hospital por día durante el periodo del estudio.
* **X5**:Número de enfermerasNúmero promedio de enfermeras, equivalentes a tiempo completo, durante el periodo
del estudio.

Se pide lo siguiente:

1. Emplee el análisis de regresión lineal múltiple que explique el riesgo de infección en términos de las variables restantes (actuando como predictoras $X_i$).
2. Identifique observaciones que puedan considerarse problemáticas (datos atípicos, puntos de balanceo e influyentes) y analice si debe eliminarlas de su conjunto de datos o no, justifique. 
3. Repita la construcción del modelo de regresión si eliminó observaciones.
4. Realice la prueba de significancia del modelo, interprete.
5. Obtener el coeficiente de determinación y el coeficiente de determinación ajustado. Interprete.
6. Analice si hay problemas de multicolinealidad.
7. Realice una selección e variables por el método que prefiera, tome decisiones, explique.
8. Realice una predicción utilizando el modelo seleccionado, interprete.

### Carga y analisis simple de la base de datos

In [3]:
# Leer archivo csv

raw_data_url = "https://raw.githubusercontent.com/repos-especializacion-UdeA/estadistica/refs/heads/main/trabajo3/Datos.csv"

# Leer el archivo CSV
df = pd.read_csv(raw_data_url)

# Mostrar las primeras filas del DataFrame
df.head()

,Y,X1,X2,X3,X4,X5
0,4.8,9.84,62.2,12.0,82.3,600
1,5.0,11.03,49.9,19.7,102.1,318
2,4.4,11.65,54.5,18.6,96.1,248
3,3.7,8.48,51.1,12.1,92.8,166
4,5.5,11.08,50.2,18.6,63.6,387


### Funciones resumen

In [4]:
df.shape

(59, 6)

In [5]:
# Información resumida de los datos numericos
df.describe()

,Y,X1,X2,X3,X4,X5
count,59.000000,59.00000,59.000000,59.000000,59.000000,59.000000
mean,4.335593,9.72678,53.452542,16.128814,81.449153,281.525424
std,1.237735,1.76993,4.023652,10.162525,19.129192,217.963723
min,1.300000,7.13000,42.000000,1.900000,39.600000,29.000000
25%,3.700000,8.51000,51.200000,8.700000,70.100000,114.000000
50%,4.400000,9.44000,53.800000,15.700000,82.500000,195.000000
75%,5.000000,10.75500,56.150000,20.700000,92.550000,335.500000
max,7.600000,17.94000,62.200000,60.500000,122.800000,835.000000


In [6]:
df.corr()

,Y,X1,X2,X3,X4,X5
Y,1.000000,0.514586,-0.138757,0.442119,0.433135,0.460360
X1,0.514586,1.000000,0.092463,0.305234,0.334557,0.517463
X2,-0.138757,0.092463,1.000000,-0.279236,-0.115365,-0.090632
X3,0.442119,0.305234,-0.279236,1.000000,0.459671,0.192034
X4,0.433135,0.334557,-0.115365,0.459671,1.000000,0.059914
X5,0.460360,0.517463,-0.090632,0.192034,0.059914,1.000000


### Estimación del modelo

El modelo se obtiene emplando el método de ajuste por mínimos cuadrados ordinarios (OLS).

In [7]:
# Para ajustar el modelo utilizando el modo fórmula, es necesario que los datos
# estén almacenados en un único dataframe.

model = smf.ols(
     formula = 'Y ~ X1 + X2 + X3 + X4 + X5',
     data = df
).fit()
display(model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.448
Model:                            OLS   Adj. R-squared:                  0.396
Method:                 Least Squares   F-statistic:                     8.593
Date:                Thu, 03 Oct 2024   Prob (F-statistic):           5.17e-06
Time:                        22:35:49   Log-Likelihood:                -78.282
No. Observations:                  59   AIC:                             168.6
Df Residuals:                      53   BIC:                             181.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.5143      1.898      0.798      0.429      -2.294       5.322
X1             0.1628      0.093      1.757      0.085      -0.023       0.349
X2            -0.0165      0.034     -0.487      0.628      -0.084       0.051
X3             0.0230      0.015      1.544      0.129      -0.007       0.053
X4             0.0159      0.008      2.042      0.046       0.000       0.031
X5             0.0016      0.001      2.312      0.025       0.000       0.003
==============================================================================
Omnibus:                        0.652   Durbin-Watson:                   2.417
Prob(Omnibus):                  0.722   Jarque-Bera (JB):                0.754
Skew:                           0.222   Prob(JB):                        0.686
Kurtosis:                       2.669   Cond. No.                     5.52e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.52e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

El modelo resultante esta definido por:

$$Y = 1.5143 + 0.1628X_1 - 0.0165X_2 + 0.0230X_3 + 0.0159X_4 + 0.0016X_5$$

Ajustar el modelo consiste en estimar, a partir de los datos disponibles, los valores de los coeficientes de regresión que maximizan la verosimilitud (likelihood), es decir, los que dan lugar al modelo que con mayor probabilidad puede haber generado los datos observados.

## Bondad de ajuste del modelo

Tenemos de la columna:
* Error estándar de los residuos (Residual Standar Error, RSE): 
 
  ```
  Adj. R-squared:	0.396 
  ```

* Coeficiente de determinación  $R^2$: 
  
  ```
  R-squared:	0.448
  ```

* Valor Estadistico F y valor P :
  
  ```
  F-statistic:	8.593
  Prob (F-statistic):	5.17e-06
  ```

* Significancia del modelo F-test:
 
Los valores p (o p-values) de los predictores en un modelo de regresión se comparan con un umbral de significancia estadística, comúnmente llamado nivel de significancia (denotado como $\alpha = 0.05$). Con:
* **Predictor estadisticamente significativo**: Si $VP < \alpha$ de modo que tiene una influencia relevante en la variable dependiente.
* **Predictor estadisticamente no significativo**: Si $VP > \alpha$: no hay suficiente evidencia para afirmar que el predictor tiene un efecto significativo en la variable dependiente.

La siguiente funcion obtiene segun lo anterior las variables significativas para el modelo:

In [9]:
# Crear un DataFrame con los valores p

p_values_df = pd.DataFrame(model.pvalues, columns=['p_value'])
p_values_df 

significativo = lambda p_value, alpha=0.05: True if p_value < alpha else False
p_values_df['significancia'] = p_values_df['p_value'].apply(significativo)
p_values_df

,p_value,significancia
Intercept,0.428630,False
X1,0.084665,False
X2,0.628346,False
X3,0.128542,False
X4,0.046120,True
X5,0.024718,True


In [12]:
# Función para obtener las variables significativas
def obtener_variables_significativas(model, alpha=0.05):
    # Ajustar el modelo de regresión
    
    # Obtener las p-valores de los predictores
    p_values = model.pvalues
    
    # Filtrar las variables con p-valor menor que alpha
    variables_significativas = p_values[p_values < alpha].index.tolist()
    
    return variables_significativas

In [13]:
variables_significativas = obtener_variables_significativas(model)
variables_significativas

['X4', 'X5']

### Analisis ANOVA SST1

Hipótesis:
* **Hipótesis nula ($𝐻_0$)**: Ninguno de los predictores tiene un efecto significativo sobre $𝑌$ (todos los coeficientes de los predictores son iguales a cero).
* **Hipótesis alternativa ($𝐻_A$)**: Al menos uno de los predictores tiene un efecto significativo sobre 𝑌

**Regla de decisión**:
* Si el valor p de la prueba ANOVA es menor que el nivel de significancia ($\alpha = 0.05$), se rechaza la hipótesis nula, lo que indica que al menos uno de los predictores es significativo.
* Si el valor p es mayor que $\alpha$, no hay evidencia suficiente para afirmar que el modelo, en su conjunto, es mejor que un modelo sin predictores.

In [16]:
# Realizar el análisis ANOVA tipo 1
anova1_model = sm.stats.anova_lm(model, typ=1)

# Mostrar los resultados de ANOVA
anova1_model

,df,sum_sq,mean_sq,F,PR(>F)
X1,1.0,23.528785,23.528785,25.411970,0.000006
X2,1.0,3.111800,3.111800,3.360860,0.072382
X3,1.0,5.655434,5.655434,6.108081,0.016704
X4,1.0,2.539382,2.539382,2.742628,0.103614
X5,1.0,4.947483,4.947483,5.343467,0.024718
Residual,53.0,49.072370,0.925894,NaN,NaN


* **Significativos**: $X_1$, $X_3$
* **No significativos**:  $X_2$, $X_3$, $X_5$

In [17]:
# Realizar el análisis ANOVA tipo 2
anova2_model = sm.stats.anova_lm(model, typ=2)

# Mostrar los resultados de ANOVA
anova2_model

,sum_sq,df,F,PR(>F)
X1,2.858808,1.0,3.087619,0.084665
X2,0.219491,1.0,0.237058,0.628346
X3,2.207230,1.0,2.383891,0.128542
X4,3.861552,1.0,4.170621,0.046120
X5,4.947483,1.0,5.343467,0.024718
Residual,49.072370,53.0,NaN,NaN


* **Significativos**: $X_4$, $X_5$
* **No significativos**:  $X_1$, $X_2$, $X_3$

## Identificacion de puntos atipicos

### Distancia de Cook

In [19]:
# Calcular la distancia de Cook
influencia = model.get_influence()

# Calcular la distancia de Cook
cooks_d = influencia.cooks_distance[0]
# Calcular los valores de apalancamiento (hii)
hii_values = influencia.hat_matrix_diag
# Calcular los residuales estudentizados
rstudent_values = influencia.resid_studentized_external
# Calcular los valores DFFITS
dffits_values = influencia.dffits[0]
# Calcular los valores DFbeta
dfbeta_values = influencia.dfbetas

# Crear un DataFrame con todos los valores de influencia
influence_df = pd.DataFrame({
    'Leverage (hii)': hii_values,
    'Cooks Distance': cooks_d,
    'R-Student': rstudent_values,
    'DFFITS': dffits_values
    # Puedes agregar más métricas si lo deseas
})

# Mostrar el DataFrame
display(influence_df)

,Leverage (hii),Cooks Distance,R-Student,DFFITS
0,0.180360,0.001202,0.179408,0.084159
1,0.057051,0.000065,-0.079473,-0.019548
2,0.051793,0.002259,-0.494597,-0.115594
3,0.050941,0.001412,-0.394105,-0.091306
4,0.077732,0.014745,1.025023,0.297581
5,0.064841,0.026946,-1.546928,-0.407336
6,0.109914,0.002620,-0.353848,-0.124345
7,0.126799,0.019050,0.885394,0.337393
8,0.027164,0.000139,-0.171543,-0.028665
9,0.040716,0.001995,0.527434,0.108662


Residuales estudentizados (mejor criterio en regresión):
Los residuales estudentizados o r-student son residuales que han sido estandarizados por su varianza, lo que permite una mejor comparación entre diferentes observaciones. Se utilizan frecuentemente para identificar outliers en modelos de regresión.

Criterio:
* Un residual estudentizado mayor a 2 o menor a -2 es considerado sospechoso.
* Un residual estudentizado mayor a 3 o menor a -3 se considera un outlier significativo.
* En modelos con grandes muestras, el umbral de ±2 suele ser suficiente para identificar posibles outliers.

In [20]:
influence_df['Outlier'] = influence_df['R-Student'].apply(lambda x: True if abs(x) > 2 else False)
influence_df

,Leverage (hii),Cooks Distance,R-Student,DFFITS,Outlier
0,0.180360,0.001202,0.179408,0.084159,False
1,0.057051,0.000065,-0.079473,-0.019548,False
2,0.051793,0.002259,-0.494597,-0.115594,False
3,0.050941,0.001412,-0.394105,-0.091306,False
4,0.077732,0.014745,1.025023,0.297581,False
5,0.064841,0.026946,-1.546928,-0.407336,False
6,0.109914,0.002620,-0.353848,-0.124345,False
7,0.126799,0.019050,0.885394,0.337393,False
8,0.027164,0.000139,-0.171543,-0.028665,False
9,0.040716,0.001995,0.527434,0.108662,False


La distancia de Cook mide cuánto cambian los coeficientes del modelo si se elimina una observación. Un valor alto de distancia de Cook indica que el punto es influyente.
Criterio:
Un valor de distancia de Cook mayor a 1 o  4/n (donde n es el número de observaciones) se considera influyente.

In [21]:
## Puntos de balanceo

# Agregar una columna para identificar puntos influyentes según la distancia de Cook
influence_df['Influyente'] = influence_df['Cooks Distance'].apply(lambda x: True if x > 4/len(influence_df) else False)
influence_df

,Leverage (hii),Cooks Distance,R-Student,DFFITS,Outlier,Influyente
0,0.180360,0.001202,0.179408,0.084159,False,False
1,0.057051,0.000065,-0.079473,-0.019548,False,False
2,0.051793,0.002259,-0.494597,-0.115594,False,False
3,0.050941,0.001412,-0.394105,-0.091306,False,False
4,0.077732,0.014745,1.025023,0.297581,False,False
5,0.064841,0.026946,-1.546928,-0.407336,False,False
6,0.109914,0.002620,-0.353848,-0.124345,False,False
7,0.126799,0.019050,0.885394,0.337393,False,False
8,0.027164,0.000139,-0.171543,-0.028665,False,False
9,0.040716,0.001995,0.527434,0.108662,False,False


## Eliminando los puntos de influencia

### Outliers

In [24]:
oulier_data = df[influence_df['Outlier'] == True]
oulier_data 

,Y,X1,X2,X3,X4,X5
39,7.6,11.41,61.1,16.6,97.9,535


La observación 39 tiene un residual estudentizado de 2.896877, lo que indica que es un outlier significativo

In [22]:
influential_data = df[influence_df['Influyente'] == True]
influential_data 

,Y,X1,X2,X3,X4,X5
24,2.9,10.79,44.2,2.6,56.6,461
28,5.9,17.94,56.2,26.4,91.8,835
36,4.9,11.07,53.2,28.5,122.0,768
39,7.6,11.41,61.1,16.6,97.9,535
47,5.4,11.18,45.7,60.5,85.8,640


Las observaciones 24, 28, 36, 39 y 47 tienen distancias de Cook que indican que son influyentes (superan el umbral calculado).

In [28]:
# Umbral de apalancamiento para identificar puntos de balanceo
leverage_threshold = 2 * (len(df.columns)) / len(df)

# Identificar puntos de balanceo (leverage > umbral)
high_leverage_points = df[hii_values > leverage_threshold]

# Mostrar los puntos de balanceo
display(high_leverage_points)

,Y,X1,X2,X3,X4,X5
24,2.9,10.79,44.2,2.6,56.6,461
28,5.9,17.94,56.2,26.4,91.8,835
36,4.9,11.07,53.2,28.5,122.0,768
47,5.4,11.18,45.7,60.5,85.8,640


Las observaciones 24, 28, 36 y 47 tienen valores de apalancamiento elevados (mayores que el umbral calculado).

In [49]:
# Filtrar los puntos influyentes (24, 28, 36, 39 y 47)
non_influential_data = df[influence_df['Influyente'] == False]
non_influential_data

,Y,X1,X2,X3,X4,X5
0,4.8,9.84,62.2,12.0,82.3,600
1,5.0,11.03,49.9,19.7,102.1,318
2,4.4,11.65,54.5,18.6,96.1,248
3,3.7,8.48,51.1,12.1,92.8,166
4,5.5,11.08,50.2,18.6,63.6,387
5,1.7,8.09,56.9,7.6,56.9,92
6,4.5,10.05,52.0,36.7,87.5,184
7,5.7,11.20,56.5,34.5,88.9,180
8,4.0,9.20,52.2,17.5,71.1,298
9,4.2,9.06,52.8,6.9,75.9,134


In [50]:
# Ajustar el modelo nuevamente con los puntos no influyentes
new_model = smf.ols('Y ~ X1 + X2 + X3 + X4 + X5', data=non_influential_data).fit()
display(new_model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.536
Model:                            OLS   Adj. R-squared:                  0.488
Method:                 Least Squares   F-statistic:                     11.09
Date:                Thu, 03 Oct 2024   Prob (F-statistic):           3.89e-07
Time:                        23:35:59   Log-Likelihood:                -63.565
No. Observations:                  54   AIC:                             139.1
Df Residuals:                      48   BIC:                             151.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.8083      1.962      1.941      0.058      -0.136       7.752
X1             0.2799      0.101      2.784      0.008       0.078       0.482
X2            -0.0714      0.034     -2.093      0.042      -0.140      -0.003
X3             0.0352      0.017      2.107      0.040       0.002       0.069
X4             0.0077      0.008      1.007      0.319      -0.008       0.023
X5             0.0019      0.001      2.758      0.008       0.001       0.003
==============================================================================
Omnibus:                        1.307   Durbin-Watson:                   2.492
Prob(Omnibus):                  0.520   Jarque-Bera (JB):                1.334
Skew:                           0.326   Prob(JB):                        0.513
Kurtosis:                       2.592   Cond. No.                     5.56e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.56e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

* Si los puntos influyentes son errores o no son representativos del fenómeno que estás estudiando, sería recomendable eliminarlos.
* Si los puntos influyentes son observaciones válidas pero extremas, no deberías eliminarlos automáticamente. Podrías optar por técnicas robustas que reduzcan su influencia.
Antes de tomar una decisión, sería útil comparar el ajuste del modelo con y sin los puntos influyentes para ver si afectan los resultados de manera significativa.

## Bondad de ajuste del modelo

Tenemos de la columna:
* Error estándar de los residuos (Residual Standar Error, RSE): 
 
  ```
  Adj. R-squared:	0.396 --> 0.488
  ```

* Coeficiente de determinación  $R^2$: 
  
  ```
  R-squared:	0.448 --> 0.536
  ```

* Valor Estadistico F y valor P :
  
  ```
  F-statistic:	8.593 --> 11.09
  Prob (F-statistic):	5.17e-06 --> 3.89e-07
  ```



1.	Coeficientes y significancia:
    * Constante: 3.8083 (p = 0.058), marginalmente significativo.
    * $X_1$: 0.2799 (p = 0.008), significativo. Cada aumento de una unidad en X1X_1X1 incrementa el riesgo de infección en 0.2799 unidades.
    * $X_2$: -0.0714 (p = 0.042), significativo. Cada aumento de una unidad en X2X_2X2 reduce el riesgo de infección en 0.0714 unidades.
    * $X_3$: 0.0352 (p = 0.040), significativo. Cada aumento en X3X_3X3 incrementa el riesgo de infección en 0.0352 unidades.
    * $X_4$: 0.0077 (p = 0.319), no significativo.
    * $X_5$: 0.0019 (p = 0.008), significativo.
2.	Ajuste del modelo:
    * $R^2 = 0.536$: El modelo explica el $53.6%$ de la variabilidad en el riesgo de infección.
    * F-statistic = 11.09, p = 3.89e-07: El modelo es significativo en su conjunto.

Conclusión:
* Las variables $X_1$, $X_2$, $X_3$, y $X_5$ tienen un impacto significativo en el riesgo de infección.
* $X_4$ no es significativa en este nuevo modelo.

Este nuevo modelo tiene un mejor ajuste en comparación con el anterior después de eliminar las observaciones problemáticas.


## 3. Realice la prueba de significancia del modelo, interprete.

In [51]:
# Global
# Obtener el valor F-statistic
f_statistic = new_model.fvalue

# Obtener el valor p asociado a la F-statistic
p_value_f = new_model.f_pvalue

# Mostrar los resultados
print(f"F-statistic: {f_statistic}")
print(f"p-value (F-statistic): {p_value_f}")

F-statistic: 11.094998013321183
p-value (F-statistic): 3.8874738191801506e-07


#### Prueba de significancia global (F-test)

* Evalúa si el modelo, en conjunto, es estadísticamente significativo.
* La hipótesis nula ($H_0$) en la prueba F es que todos los coeficientes de las variables independientes son iguales a cero. Es decir, las variables predictoras no tienen efecto sobre la variable dependiente.
* La hipótesis alternativa ($H_A$) es que al menos uno de los coeficientes es diferente de cero, lo que implica que las variables independientes tienen un efecto significativo en la variable dependiente.

En este modelo:
* La F-statistic es 11.09 con un valor p = 3.89e-07, lo que significa que el modelo es globalmente significativo.
* Esto indica que, en conjunto, las variables predictoras (X1,X2,X3,X4,X5) explican significativamente la variabilidad en la variable dependiente (Y).

In [52]:
## Anova 1

# Realizar el análisis ANOVA tipo 1
anova1_new_model  = sm.stats.anova_lm(new_model, typ=1)

# Mostrar los resultados de ANOVA
anova1_new_model 

,df,sum_sq,mean_sq,F,PR(>F)
X1,1.0,19.672845,19.672845,28.362681,0.000003
X2,1.0,10.023632,10.023632,14.451245,0.000406
X3,1.0,3.394647,3.394647,4.894121,0.031740
X4,1.0,0.111029,0.111029,0.160072,0.690865
X5,1.0,5.276257,5.276257,7.606871,0.008202
Residual,48.0,33.293628,0.693617,NaN,NaN


* Si: X1, X2, X3, X5
* No: X4

In [53]:
## Anova 2

# Realizar el análisis ANOVA tipo 1
anova2_new_model  = sm.stats.anova_lm(new_model, typ=2)

# Mostrar los resultados de ANOVA
anova2_new_model 

,sum_sq,df,F,PR(>F)
X1,5.376582,1.0,7.751511,0.007656
X2,3.037081,1.0,4.378612,0.041704
X3,3.078608,1.0,4.438483,0.040390
X4,0.703998,1.0,1.014966,0.318768
X5,5.276257,1.0,7.606871,0.008202
Residual,33.293628,48.0,NaN,NaN


* Si: X1, X2, X3, X5
* No: X4

### 4. Coeficiente de determinación y el Coeficiente de determinación ajustado

Ya de obtuvo del resultado previo

In [54]:
# Obtener el coeficiente de determinación (R^2)
r_squared = new_model.rsquared

# Obtener el coeficiente de determinación ajustado (R^2 ajustado)
r_squared_adj = new_model.rsquared_adj

# Mostrar los resultados
print(f"R^2: {r_squared}")
print(f"R^2 ajustado: {r_squared_adj}")

R^2: 0.5361197911775315
R^2 ajustado: 0.48779893609185765


Si el $R^2$ de tu modelo es 0.536, significa que el 53.6% de la variabilidad en el riesgo de infección es explicado por las variables predictoras en el modelo. Si el $R^2$ ajustado es 0.488, significa que este valor ha sido penalizado debido a la inclusión de predictores adicionales, indicando que el modelo aún tiene un buen ajuste, pero con algunas variables que pueden no estar añadiendo mucho valor.
Conclusión:

* $R^2$ te dice cuánta variabilidad del resultado está explicada por los predictores.
* $R^2$ ajustado corrige el regular para evitar que se vea artificialmente inflado por predictores adicionales no significativos

## 5. Analisis de problemas de multicolinealidad

In [59]:
### Factores de Inflación de Varianza

# Agregar constante si es necesario
X_non_influential_data = non_influential_data[['X1','X2','X3','X4','X5']]
X_with_const = sm.add_constant(X_non_influential_data)

# Calcular VIF para cada variable predictora
vif = pd.DataFrame()
vif["Variable"] = X_non_influential_data.columns
vif["VIF"] = [variance_inflation_factor(X_with_const.values, i) for i in range(1, X_with_const.shape[1])]

# Mostrar los resultados
print(vif)

  Variable       VIF
0       X1  1.498908
1       X2  1.220344
2       X3  1.420350
3       X4  1.553573
4       X5  1.250247


A continuación se establece el criterio para detectar la
multicolinealidad de acuerdo a esta medida.
* Si VIFj < 5 no hay multicolinealidad.
* Si 5 < VIFj < 10 hay multicolinealidad moderada.
* Si VIFj < 10 hay multicolinealidad grave.

Parece que no hay colinealidad.

### Matrix de correlacion

In [60]:
corr_matrix = X_non_influential_data.corr()
print(corr_matrix)

          X1        X2        X3        X4        X5
X1  1.000000  0.094926  0.253783  0.360761  0.339257
X2  0.094926  1.000000 -0.290758 -0.231524 -0.087547
X3  0.253783 -0.290758  1.000000  0.493866 -0.029846
X4  0.360761 -0.231524  0.493866  1.000000 -0.073206
X5  0.339257 -0.087547 -0.029846 -0.073206  1.000000


### Análisis de los valores propios

In [63]:
# Obtener los eigenvalores de la matriz de correlación
eigvals, eigvecs = np.linalg.eig(corr_matrix)

### Número de condición

In [70]:
# Calcular el número de condición (Condition Number)
condition_number = np.linalg.cond(X_non_influential_data)

# Mostrar el número de condición
print(condition_number)
print(np.sqrt(condition_number)) # hay multicolinealidad moderada

286.74465349823333
16.933536355358065


El número de condición obtenido fue 286.74, lo cual es considerablemente alto y sugiere que las variables predictoras están fuertemente correlacionadas.

### Índice de condición:

In [69]:
# Calcular el índice de condición
condition_index = np.sqrt(eigvals.max() / eigvals)
print("Índice de condición:", condition_index)

Índice de condición: [1.         1.21159131 1.36880053 2.21698016 1.90558112]


Todos los índices de condición están por debajo de 10, lo que indica que no hay multicolinealidad severa. Aunque el número de condición general era elevado, los índices de condición individuales no indican problemas significativos de multicolinealidad entre las variables.

### Proporsiones de descomposicion de variables

In [72]:
# Calcular las proporciones de descomposición de la varianza
proportion_var_decomposition = pd.DataFrame((eigvecs**2) / eigvals, columns=X_non_influential_data.columns)

# Mostrar las proporciones de descomposición de la varianza
proportion_var_decomposition

,X1,X2,X3,X4,X5
0,0.104001,0.257640,0.073612,1.014301,0.049354
1,0.055732,0.107239,0.558989,0.356232,0.142152
2,0.179371,0.041055,0.004339,0.002025,1.193559
3,0.189668,0.019599,0.070927,0.733911,0.539468
4,0.009986,0.365341,0.301559,0.541532,0.031829


¿Qué puedes observar?
Componentes asociados con índices elevados (si se calculan) deben ser revisados. Si alguna variable tiene una proporción de descomposición alta (generalmente mayor a 0.5) en esos componentes, significa que está contribuyendo significativamente a la multicolinealidad.

En este caso, podemos ver que para el componente 3, la variable X5 tiene una proporción de 1.193, lo que indica una contribución significativa a la colinealidad en este componente.

Conclusión:
La variable X5 parece estar contribuyendo considerablemente a la multicolinealidad en el modelo, ya que tiene proporciones de descomposición altas en varios componentes.
Otras variables, como X4 y X3, también muestran proporciones elevadas en algunos componentes, lo que sugiere que también podrían estar contribuyendo a la colinealidad.

### 6. Selección de variables 

In [73]:
def todas_regresiones_posibles_completo(data, respuesta):
    variables_predictoras = list(data.columns)
    variables_predictoras.remove(respuesta)
    
    n = len(data)
    resultados = []

    # Probar todas las combinaciones de variables predictoras
    for k in range(1, len(variables_predictoras) + 1):
        for combo in itertools.combinations(variables_predictoras, k):
            # Definir el conjunto de predictores actual
            X = data[list(combo)]
            X = sm.add_constant(X)  # Agregar la constante (intercepto)
            y = data[respuesta]
            
            # Ajustar el modelo de regresión
            modelo = sm.OLS(y, X).fit()
            
            # Obtener los valores necesarios
            R2 = modelo.rsquared
            R2_adj = modelo.rsquared_adj
            SSE = np.sum(modelo.resid ** 2)  # Suma de cuadrados del error
            MSE = SSE / modelo.df_resid  # Error cuadrático medio
            Cp = SSE / MSE - (n - 2 * (k + 1))  # Criterio de Cp de Mallows
            
            # Guardar los resultados del modelo
            resultados.append({
                'Predictoras': combo,
                'R2': R2,
                'R2_adj': R2_adj,
                'SSE': SSE,
                'MSE': MSE,
                'Cp': Cp,
                'AIC': modelo.aic,
                'BIC': modelo.bic
            })
    
    # Convertir los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados)
    
    return resultados_df

In [76]:
# Ejemplo de uso (suponiendo que df es tu DataFrame)
resultados_posibles = todas_regresiones_posibles_completo(non_influential_data, 'Y')
resultados_posibles

,Predictoras,R2,R2_adj,SSE,MSE,Cp,AIC,BIC
0,"(X1,)",0.274102,0.260142,52.099192,1.001908,2.0,155.310293,159.288261
1,"(X2,)",0.103893,0.086660,64.315413,1.236835,2.0,166.685386,170.663354
2,"(X3,)",0.202374,0.187035,57.247221,1.100908,2.0,160.398695,164.376663
3,"(X4,)",0.157333,0.141128,60.479936,1.163076,2.0,163.365054,167.343022
4,"(X5,)",0.176686,0.160853,59.090891,1.136363,2.0,162.110369,166.088337
5,"(X1, X2)",0.413761,0.390771,42.075560,0.825011,3.0,145.771444,151.738396
6,"(X1, X3)",0.381503,0.357249,44.390756,0.870407,3.0,148.663911,154.630863
7,"(X1, X4)",0.323732,0.297212,48.537108,0.951708,3.0,153.485968,159.452920
8,"(X1, X5)",0.340679,0.314824,47.320789,0.927859,3.0,152.115508,158.082460
9,"(X2, X3)",0.242443,0.212735,54.371396,1.066106,3.0,159.615486,165.582438


In [77]:
# R2_adj
resultados_posibles.sort_values(by='R2_adj', ascending=False)

,Predictoras,R2,R2_adj,SSE,MSE,Cp,AIC,BIC
30,"(X1, X2, X3, X4, X5)",0.536120,0.487799,33.293628,0.693617,6.0,139.129988,151.063892
26,"(X1, X2, X3, X5)",0.526311,0.487642,33.997625,0.693829,5.0,138.259920,148.204840
28,"(X1, X3, X4, X5)",0.493804,0.452482,36.330708,0.741443,5.0,141.844043,151.788964
27,"(X1, X2, X4, X5)",0.493226,0.451856,36.372236,0.742291,5.0,141.905733,151.850653
19,"(X1, X3, X5)",0.472859,0.441230,37.834018,0.756680,4.0,142.033490,149.989426
15,"(X1, X2, X3)",0.461059,0.428722,38.680913,0.773618,4.0,143.228922,151.184858
17,"(X1, X2, X5)",0.458695,0.426217,38.850536,0.777011,4.0,143.465204,151.421140
25,"(X1, X2, X3, X4)",0.462606,0.418737,38.569884,0.787140,5.0,145.073699,155.018619
29,"(X2, X3, X4, X5)",0.461208,0.417225,38.670209,0.789188,5.0,145.213977,155.158897
24,"(X3, X4, X5)",0.443768,0.410394,39.921898,0.798438,4.0,144.934173,152.890109


### Posible modelo

In [82]:
# Definir las variables predictoras (X1) y la variable dependiente
X1 = X_non_influential_data[['X1']]  # Usamos solo la variable X1
X1 = sm.add_constant(X1)  # Agregar el intercepto
Y = non_influential_data[['Y']]

# Ajustar el modelo de regresión
model_X1 = sm.OLS(Y, X1).fit()

# Mostrar el resumen del modelo
print(model_X1.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.274
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     19.64
Date:                Fri, 04 Oct 2024   Prob (F-statistic):           4.86e-05
Time:                        00:30:17   Log-Likelihood:                -75.655
No. Observations:                  54   AIC:                             155.3
Df Residuals:                      52   BIC:                             159.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1000      0.945      0.106      0.9

In [81]:
# Definir las variables predictoras (X1) y la variable dependiente
X3 = X_non_influential_data[['X1', 'X2', 'X3', 'X5']]  # Usamos solo la variable X1
X3 = sm.add_constant(X3)  # Agregar el intercepto
Y = non_influential_data[['Y']]

# Ajustar el modelo de regresión
model_X3 = sm.OLS(Y, X3).fit()

# Mostrar el resumen del modelo
print(model_X3.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.526
Model:                            OLS   Adj. R-squared:                  0.488
Method:                 Least Squares   F-statistic:                     13.61
Date:                Fri, 04 Oct 2024   Prob (F-statistic):           1.56e-07
Time:                        00:27:45   Log-Likelihood:                -64.130
No. Observations:                  54   AIC:                             138.3
Df Residuals:                      49   BIC:                             148.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.4038      1.871      2.354      0.0

In [84]:
# Definir las variables predictoras (X1) y la variable dependiente
X2 = X_non_influential_data[['X1','X2']]  # Usamos solo la variable X1
X2 = sm.add_constant(X2)  # Agregar el intercepto
Y = non_influential_data[['Y']]

# Ajustar el modelo de regresión
model_X1_X2 = sm.OLS(Y, X2).fit()

# Mostrar el resumen del modelo
print(model_X1_X2.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.414
Model:                            OLS   Adj. R-squared:                  0.391
Method:                 Least Squares   F-statistic:                     18.00
Date:                Fri, 04 Oct 2024   Prob (F-statistic):           1.22e-06
Time:                        00:31:30   Log-Likelihood:                -69.886
No. Observations:                  54   AIC:                             145.8
Df Residuals:                      51   BIC:                             151.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.1409      1.934      3.176      0.0

## 7. Prediccion

In [86]:
# Crear un nuevo conjunto de datos para predicción (pueden ser valores nuevos o los mismos usados en el ajuste)
X_new = pd.DataFrame({
    'X1': [5, 10, 15],  # Ejemplo de valores nuevos para X1
    'X2': [2, 4, 6]     # Ejemplo de valores nuevos para X2
})

# Agregar la constante
X_new = sm.add_constant(X_new)

# Realizar la predicción
predictions = model_X1_X2.predict(X_new)

# Mostrar las predicciones
predictions

0     8.240454
1    10.340010
2    12.439566
dtype: float64

**Referencias**:
1. https://cienciadedatos.net/documentos/py20-clustering-con-python
2. https://cienciadedatos.net/documentos/pystats06-analisis-normalidad-python
3. https://cienciadedatos.net/documentos/pystats09-analisis-de-varianza-anova-python
4. https://cienciadedatos.net/documentos/pystats05-correlacion-lineal-python
5. https://cienciadedatos.net/documentos/py10b-regresion-lineal-multiple-python
6. https://jakevdp.github.io/PythonDataScienceHandbook/
7. https://datatofish.com/multiple-linear-regression-python/
8. https://www.kaggle.com/code/emineyetm/multiple-linear-regression-in-python
9. https://rpubs.com/emlopezr/E2Informe2
10. https://rpubs.com/hsajona/805025
11. https://rpubs.com/Jaolarteh/802574
12. https://datatofish.com/statsmodels-linear-regression/
13. https://fhernanb.github.io/
14. https://fhernanb.github.io/cur_est2.html
15. https://fhernanb.github.io/books.html